# Code for Cahn-Hilliard phase separation with mechanical coupling.


## 2D phase separation study.




Degrees of freedom: 
  - scalar chemical potential: we use normalized  mu = mu/RT 
  - species concentration:  we use normalized  c= Omega*cmat 
  
### Units:
- Length: um
- Mass: kg
- Time: s
- Amount of substance: pmol
- Temperature: K
- Mass density: kg/um^3
- Force: uN
- Stress: MPa
- Energy: pJ
- Species concentration: pmol/um^3
- Chemical potential: pJ/pmol
- Molar volume: um^3/pmol
- Species diffusivity: um^2/s
- Boltzmann Constant: 1.38E-11 pJ/K
- Gas constant: 8.314  pJ/(pmol K)

### By
  Eric Stewart      and      Lallit Anand
ericstew@mit.edu            anand@mit.edu

October 2023

Modified for FenicsX by Jorge Nin
jorgenin@mit.edu

In [1]:
import numpy as np


from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log, default_scalar_type
from dolfinx.fem import Constant, dirichletbc, Function, FunctionSpace, Expression
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter

import ufl
from ufl import (
    TestFunction,
    TrialFunction,
    Identity,
    grad,
    det,
    div,
    dev,
    inv,
    tr,
    sqrt,
    conditional,
    gt,
    dx,
    inner,
    derivative,
    dot,
    ln,
    split,
    tanh,
    as_tensor,
    as_vector,
    ge
)

from hilliard_models import Cahn_Hillard_Plane_Strain
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista

pyvista.set_jupyter_backend("client")
## Define temporal parameters
import random

# DEFINE GEOMETRY

In [2]:
problemName = "Canh Hillard Mechanical"

# Square edge length
L0 = 0.8  # 800 nm box, after Di Leo et al. (2014)

# Number of elements along each side
N = 30

# Create square mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD, [(0, 0), (L0, L0)], [N, N])

## Visualize the geometry

In [3]:

plotter = pyvista.Plotter()
vtkdata = vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(*vtkdata)
actor = plotter.add_mesh(grid, show_edges=True)
plotter.show()
plotter.close()

Widget(value="<iframe src='http://localhost:53789/index.html?ui=P_0x2a2c31190_0&reconnect=auto' style='width: …

# Periodicity

In [4]:
def inside(self, x, on_boundary):
    # return True if on left or bottom boundary AND NOT
    # on one of the two corners (0, L0) and (L0, 0)
    return bool(
        (np.isclose(x[0], 0) or np.isclose(x[1], 0))
        and (
            not (
                (np.isclose(x[0], 0) and np.isclose(x[1], L0))
                or (np.isclose(x[0], L0) and np.isclose(x[1], 0))
            )
        )
        and on_boundary
    )


def map(self, x, y):
    if np.isclose(x[0], L0) and np.isclose(x[1], L0):
        y[0] = x[0] - L0
        y[1] = x[1] - L0
    elif np.isclose(x[0], L0):
        y[0] = x[0] - L0
        y[1] = x[1]
    else:  # np.isclose(x[1], L0)
        y[0] = x[0]
        y[1] = x[1] - L0

# Simulation Time Control

In [5]:
t = 0.0  # initialization of time
Ttot = 4  # total simulation time
dt = 0.01  # Initial time step size, here we will use adaptive time-stepping

# Create Problem

In [6]:
dk = Constant(domain,dt)

hillard_problem = Cahn_Hillard_Plane_Strain(domain)
hillard_problem.Kinematics()
hillard_problem.WeakForms(dk)

# Setup Output Files

In [7]:
U1 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 1)
V2 = fem.FunctionSpace(domain, U1)#Vector function space
V1 = fem.FunctionSpace(domain, hillard_problem.P1)#Scalar function space

mu_vis = Function(V1)
mu_vis.name = "mu"
mu_expr = Expression(hillard_problem.mu,V1.element.interpolation_points())

c_vis = Function(V1)
c_vis.name = "c"
c_expr = Expression(hillard_problem.c,V1.element.interpolation_points())



def interp_and_save(t, file):

    mu_vis.interpolate(mu_expr)
    c_vis.interpolate(c_expr)

    file.write(t)


# Boundary Conditions

In [8]:
#Locate the boundary
def bottom(x):
    return np.isclose(x[1], 0)


def right(x):
    return np.isclose(x[0], 0)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, right)
bottom_facets = mesh.locate_entities_boundary(domain, fdim, bottom)

marked_facets = np.hstack([bottom_facets, left_facets])
marked_values = np.hstack([np.full_like(bottom_facets, 1), np.full_like(left_facets, 2)])
sorted_facets = np.argsort(marked_facets)

facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

In [9]:
# Just fix the sides to make sure they don't move

u_bc = np.array((0), dtype=default_scalar_type)




left_dofs = fem.locate_dofs_topological(hillard_problem.ME.sub(0).sub(0), facet_tag.dim, facet_tag.find(2)) #we don't want it to move in the x direction
bottom_dofs = fem.locate_dofs_topological(hillard_problem.ME.sub(0).sub(1), facet_tag.dim, facet_tag.find(1)) #we don't want it to move in the y direction
bcs = [fem.dirichletbc(u_bc, left_dofs, hillard_problem.ME.sub(0).sub(0)),
       fem.dirichletbc(u_bc, bottom_dofs, hillard_problem.ME.sub(0).sub(1))]


# Setup Nonlinear Problem

In [10]:
import os
step = "Swell"
jit_options ={"cffi_extra_compile_args":["-O3","-ffast-math"]}

problem = NonlinearProblem(hillard_problem.Res,hillard_problem.w, bcs, hillard_problem.a)


solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-8
solver.atol = 1e-8
solver.max_it = 50
solver.report = True
solver.error_on_nonconvergence = False

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_max_it"] = 30
opts[f"{option_prefix}ksp_type"] = "cg"
#opts[f"{option_prefix}pc_type"] = "ksp"
ksp.setFromOptions()

startTime = datetime.now()
print("------------------------------------")
print("Simulation Start")
print("------------------------------------")

step = "Evolve"

#if os.path.exists("results/"+problemName+".bp"):
#    os.remove("results/"+problemName+".xdmf")
#    os.remove("results/"+problemName+".h5")

#vtk.write_mesh(domain)
t = 0.0

interp_and_save(t, vtk)
ii = 0
bisection_count = 0
while t < Ttot:
    # increment time
    t += float(dk) 
    # increment counter
    ii +=1
    

    # Solve the problem
    
    (iter, converged) = solver.solve(hillard_problem.w)
    
    if converged:
        hillard_problem.w.x.scatter_forward()
        
        
        
        hillard_problem.w_old.x.array[:] = hillard_problem.w.x.array
        
        interp_and_save(t, vtk)
        if ii % 1 == 0:
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            print("Step: {} |   Increment: {} | Iterations: {}".format(step, ii, iter))
            print("Simulation Time: {} s | dt: {} s".format(round(t, 2), round(dt, 3)))
            print()
        
        if iter <= 2:
            dt = 1.5 * dt
            dk.value = dt
        # If the newton solver takes 5 or more iterations,
        # decrease the time step by a factor of 2:
        elif iter >= 5:
            dt = dt / 2
            dk.value =dt

        #Reset Biseciton Counter
        bisection_count = 0
        
    else:
     # Break the loop if solver fails too many times
        bisection_count += 1
        
        if bisection_count > 5:
            print("Error: Too many bisections")
            break
        
        print( "Error Halfing Time Step")
        t = t - float(dk)
        dt = dt / 2
        dk.value = dt
        print(f"New Time Step: {dt}")
        hillard_problem.w.x.array[:] = hillard_problem.w_old.x.array
        

#End Analysis
vtk.close()
endTime = datetime.now()
print("------------------------------------")
print("Simulation End")
print("------------------------------------")
print("Total Time: {}".format(endTime - startTime))
print("------------------------------------")



    
    

------------------------------------
Simulation Start
------------------------------------


NameError: name 'vtk' is not defined